# Poisoning Attack in Federated Learning Experiment 1

In [12]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [13]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')
sys.path.append('/content/drive/My Drive/Colab Notebooks/federated_learning')

In [5]:
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 23.5 MB/s 


Purpose: 
- What is the performance loss induced through the different poisoning attack? 
- What is the performance of the global model without poisoning?


In [14]:
from federated_learning.utils import SHAPUtil, experiment_util
from federated_learning import ClientPlane, Configuration, ObserverConfiguration
from federated_learning.server import Server
from datetime import datetime

## Additional Functions

In [15]:
class Experiment():
    def __init__(self, num_p_clients, from_label, to_label, natural_label ):
        self.num_p_clients = num_p_clients
        self.from_label = from_label
        self.to_label = to_label
        self.neutral_label = natural_label
        self.accuracy = []
        self.recall = {}
        self.initialize_recall()

    def initialize_recall(self):
        self.recall[self.from_label] = []
        self.recall[self.to_label] = []
        self.recall[self.neutral_label] = []
        

In [ ]:
experiments = []

In [8]:
def add_round_to_experiment(experiment, recall, accuracy, from_label, to_label, neutral_label): 
    experiment.accuracy.append(accuracy)
    experiment.recall[from_label].append(recall[from_label])
    experiment.recall[to_label].append(recall[to_label])
    experiment.recall[neutral_label].append(recall[neutral_label])

## Experiment 1.0: MNIST(5,4)

In [30]:
from federated_learning.nets import MNISTCNN
from federated_learning.dataset import MNISTDataset
import os
config = Configuration()
config.TEMP = os.path.join('/content/drive/My Drive/Colab Notebooks/temp')
config.FMNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.MNIST_DATASET_PATH = os.path.join('/content/data/fmnist')
config.CIFAR10_DATASET_PATH = os.path.join('/content/data/fmnist')
config.VM_URL = "none"
config.FROM_LABEL = 5
config.TO_LABEL = 4
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = MNISTDataset
config.MODELNAME = config.MNIST_NAME
config.NETWORK = MNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.datasetObserverConfiguration = "MNIST"
neutral_label = 2

In [28]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 

MNIST training data loaded.
MNIST test data loaded.


In [31]:
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)

Create 200 clients with dataset of size 300


In [ ]:
import numpy as np
import copy
for num_p_clients in [0, 10, 25, 50,75, 100, 200]:
    client_plane.reset_default_client_nets()
    server.reset_to_default_net()
    client_plane.reset_poisoning_attack()
    config.POISONED_CLIENTS = num_p_clients
    experiment = Experiment(num_p_clients, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    server.test()
    recall, precision, accuracy = server.analize_test()
    add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    for i in range(200):
        experiment_util.set_rounds(client_plane, server, i+1)
        experiment_util.run_round(client_plane, server, i+1)
        if (i+1)%10 == 0:
            server.test()
            recall, precision, accuracy = server.analize_test()
            add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
            print("Round {} finished".format(i+1))
    print(experiment.accuracy)
    print(experiment.recall[config.FROM_LABEL])
    print(experiment.recall[config.TO_LABEL])
    print(experiment.recall[natural_label])
    experiments.append(copy.deepcopy(experiment))

Load default model successfully
No poisoning due to 0. poisoned clients

Test set: Average loss: 0.0023, Accuracy: 636/10000 (6%)


Test set: Average loss: 0.0013, Accuracy: 7977/10000 (80%)

Round 10 finished

Test set: Average loss: 0.0005, Accuracy: 9080/10000 (91%)

Round 20 finished

Test set: Average loss: 0.0004, Accuracy: 9271/10000 (93%)

Round 30 finished

Test set: Average loss: 0.0003, Accuracy: 9382/10000 (94%)

Round 40 finished
Model aggregation in round 50 was successful

Test set: Average loss: 0.0003, Accuracy: 9460/10000 (95%)

Round 50 finished

Test set: Average loss: 0.0002, Accuracy: 9516/10000 (95%)

Round 60 finished

Test set: Average loss: 0.0002, Accuracy: 9557/10000 (96%)

Round 70 finished

Test set: Average loss: 0.0002, Accuracy: 9560/10000 (96%)

Round 80 finished

Test set: Average loss: 0.0002, Accuracy: 9602/10000 (96%)

Round 90 finished
Model aggregation in round 100 was successful
Train Epoch: 100 [0/300 (0%)]	Loss: 0.013946
Train Epoch: 100 [10/3

In [ ]:
for e in experiments: 
    print(e.accuracyracy)
    print(e.recall[config.FROM_LABEL])
    print(e.recall[config.TO_LABEL])
    print(e.recall[neutral_label])

# Experiment 1.0: Fashion MNIST(5,4)

In [16]:
from federated_learning.nets import FMNISTCNN
from federated_learning.dataset import FMNISTDataset
config = Configuration()
config.FROM_LABEL = 5
config.TO_LABEL = 4
config.POISONED_CLIENTS = 0
config.DATA_POISONING_PERCENTAGE = 1
config.DATASET = FMNISTDataset
config.MODELNAME = config.FMNIST_NAME
config.NETWORK = FMNISTCNN
observer_config = ObserverConfiguration()
observer_config.experiment_type = "shap_fl_poisoned"
observer_config.experiment_id = 1
observer_config.test = False
observer_config.dataset = "FMNIST"
neutral_label = 2

In [17]:
data = config.DATASET(config)
shap_util = SHAPUtil(data.test_dataloader) 
server = Server(config, observer_config,data.train_dataloader, data.test_dataloader, shap_util)
client_plane = ClientPlane(config, observer_config, data, shap_util)
experiments_fmnist = []

FashionMnist training data loaded.
FashionMnist training data loaded.


AttributeError: ignored

In [ ]:
import numpy as np
import copy
for num_p_clients in [0, 10, 25, 50, 75, 100, 200]:
    client_plane.reset_default_client_nets()
    server.reset_to_default_net()
    client_plane.reset_poisoning_attack()
    config.POISONED_CLIENTS = num_p_clients
    experiment = Experiment(num_p_clients, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    experiment_util.update_configs(client_plane, server, config, observer_config)
    client_plane.poison_clients()
    server.test()
    recall, precision, accuracy = server.analize_test()
    add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
    for i in range(200):
        experiment_util.set_rounds(client_plane, server, i+1)
        experiment_util.run_round(client_plane, server, i+1)
        if (i+1)%10 == 0:
            server.test()
            recall, precision, accuracy = server.analize_test()
            add_round_to_experiment(experiment, recall, accuracy, config.FROM_LABEL, config.TO_LABEL, neutral_label)
            print("Round {} finished".format(i+1))
    print(experiment.accuracy)
    print(experiment.recall[config.FROM_LABEL])
    print(experiment.recall[config.TO_LABEL])
    print(experiment.recall[natural_label])
    experiments_fmnist.append(copy.deepcopy(experiment))

Load default model successfully
No poisoning due to 0. poisoned clients


dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).



Test set: Average loss: 0.0023, Accuracy: 1252/10000 (13%)


Test set: Average loss: 0.0005, Accuracy: 8162/10000 (82%)

Round 10 finished

Test set: Average loss: 0.0005, Accuracy: 8411/10000 (84%)

Round 20 finished

Test set: Average loss: 0.0004, Accuracy: 8540/10000 (85%)

Round 30 finished
